Latest update: 2022-11-21 15:39:25.892880

<a href="https://colab.research.google.com/github/ocean-data-factory-sweden/kso-data-management/blob/main/tutorials/01_Check_and_update_csv_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">Colab KSO Tutorial #1: Check and update csv files</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>

# Set up and requirements

## Install kso_data_management and its requirements

In [7]:
from IPython.display import clear_output

try:
    import google.colab
    import os

    IN_COLAB = True
    print("Running in Colab...")

    # Clone kso-data-management repo
    !git clone --quiet --recurse-submodules -b main https://github.com/ocean-data-factory-sweden/kso-data-management.git
    !pip install -q --upgrade pip
    !pip install -q -r kso-data-management/requirements.txt

    # Fix libmagic issue
    !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

    # Enable external widgets
    from google.colab import output

    output.enable_custom_widget_manager()

    os.chdir("kso-data-management/tutorials")
    try:
        clear_output()
        print("All packages are installed and ready to go!")
    except:
        clear_output()
        print("There have been some issues installing the packages!")
except:
    IN_COLAB = False
    import sys

    # Install requirements
    !pip install -q --no-warn-script-location --upgrade pip
    !pip install -qr ../requirements.txt

    !jupyter nbextension install --user --py widgetsnbextension
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension install --user --py jupyter_bbox_widget
    !jupyter nbextension enable --user --py jupyter_bbox_widget

    clear_output()
    print("Running locally... you're good to go!")

Running locally... you're good to go!


### Import Python packages

In [8]:
# Set the directory of the libraries
import sys, os
from pathlib import Path

# Enables testing changes in utils
%load_ext autoreload
%autoreload 2

# Specify the path of the tutorials
sys.path.append("..")

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.project_utils as p_utils
import kso_utils.server_utils as s_utils
import kso_utils.t1_utils as t1

print("Packages loaded successfully")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Packages loaded successfully


### Choose your project

In [9]:
project_name = t_utils.choose_project()

Dropdown(description='Project:', options=('Template project', 'Koster_Seafloor_Obs', 'Spyfish_Aotearoa', 'SGU'…

## Initiate database

In [10]:
project = p_utils.find_project(project_name=project_name.value)
db_info_dict = t_utils.initiate_db(project)

INFO:root:Template project loaded succesfully
INFO:root:Retrieving the file from https://drive.google.com/uc?&confirm=s5vl&id=1PZGRoSY_UpyLfMhRphMUMwDXw4yx1_Fn
Downloading...
From: https://drive.google.com/uc?&confirm=s5vl&id=1PZGRoSY_UpyLfMhRphMUMwDXw4yx1_Fn
To: /Users/jurie.germishuys/Workspace/odf/koster-uw/database/kso_data_management/tutorials/db_csv_info.zip
100%|██████████| 2.29k/2.29k [00:00<00:00, 2.39MB/s]
INFO:root:Updated sites
INFO:root:Updated movies
INFO:root:Updated species


# Review Sites

## Map sites and metadata

In [11]:
kso_map = t1.map_site(db_info_dict, project)
kso_map

## Manually update sites metadata

Select the range of sites to display

In [ ]:
sites_df, sites_range = t1.select_sheet_range(
    db_info_dict=db_info_dict, orig_csv="local_sites_csv"
)

Update the contents of the cells in the spreadsheet below as needed

In [ ]:
sites_df_filtered, sites_sheet = t1.open_csv(df=sites_df, df_range=sites_range)
sites_sheet

### Review and confirm the changes

In [ ]:
# Display the changes
highlight_changes, sites_sheet_df = t1.display_changes(
    db_info_dict, isheet=sites_sheet, df_filtered=sites_df_filtered
)
highlight_changes

In [ ]:
# Confirm or deny the changes
t1.update_csv(
    db_info_dict,
    project,
    sheet_df=sites_sheet_df,
    df=sites_df,
    local_csv="local_sites_csv",
    serv_csv="server_sites_csv",
)

# Review Movies

## Preview movies

### Retrieve info of movies available on the server

In [ ]:
available_movies_df = s_utils.retrieve_movie_info_from_server(
    project=project, db_info_dict=db_info_dict
)

### Select movies to preview

In [ ]:
movie_selected = t_utils.select_movie(available_movies_df)

### Display the movie

In [ ]:
movie_display, movie_path = t_utils.preview_movie(
    project=project,
    db_info_dict=db_info_dict,
    available_movies_df=available_movies_df,
    movie_i=movie_selected.value,
)
movie_display

## Automatic check movies metadata

 Choose the review method

In [ ]:
review_method = t1.choose_movie_review()

Specify the GPU availability

In [ ]:
gpu_available = t_utils.gpu_select()

Check and update the movies information

In [ ]:
t1.check_movies_csv(db_info_dict, project, review_method, gpu_available.result)

## Manually update movies metadata

Select the range of movies to display

In [ ]:
movies_df, movies_range = t1.select_sheet_range(
    db_info_dict=db_info_dict, orig_csv="local_movies_csv"
)

Update the contents of the cells in the spreadsheet below as needed

In [ ]:
movies_df_filtered, movies_sheet = t1.open_csv(df=movies_df, df_range=movies_range)
movies_sheet

### Review and confirm the changes

In [ ]:
# Display the changes
highlight_changes, movies_sheet_df = t1.display_changes(
    db_info_dict, isheet=movies_sheet, df_filtered=movies_df_filtered
)
highlight_changes

In [ ]:
# Confirm or deny the changes
t1.update_csv(
    db_info_dict,
    project,
    sheet_df=movies_sheet_df,
    df=movies_df,
    local_csv="local_movies_csv",
    serv_csv="server_movies_csv",
)

# Review species

## Check the species dataframe

In [ ]:
species_sheet_df = t1.check_species_csv(db_info_dict=db_info_dict, project=project)

## Manually update species dataframe

Select the range of species to display

In [ ]:
species_df, species_range = t1.select_sheet_range(
    db_info_dict=db_info_dict, orig_csv="local_species_csv"
)

Update the contents of the cells in the spreadsheet below as needed

In [ ]:
species_df_filtered, species_sheet = t1.open_csv(df=species_df, df_range=species_range)
species_sheet

### Review and confirm the changes

In [ ]:
# Display the changes
highlight_changes, species_sheet_df = t1.display_changes(
    db_info_dict, isheet=species_sheet, df_filtered=species_df_filtered
)
highlight_changes

In [ ]:
# Confirm or deny the changes
t1.update_csv(
    db_info_dict,
    project,
    sheet_df=species_sheet_df,
    df=species_df,
    local_csv="local_species_csv",
    serv_csv="server_species_csv",
)

In [ ]:
# END